<div class="row">
  <div class="col-sm-8">
     <center>
        <h1>
        IGR 204 - Visualisation<br>
        Mini-projet (Prénoms en France)<br>
        PARTIE 1 - Evolution dans le temps
            <br>
            <br>
            <center><img src="https://s3-eu-west-1.amazonaws.com/assets.atout-on-line.com/images/ingenieur/Fiches_Ecoles/telecom_paristech.jpg" alt="drawing" width="170"/></center>
        <hr class="bg-light" />
            <i>Léa PAPILLON, Pierrick LEROY, Yann RODRIGUEZ, Ramzi KHALIFA, Alexandre LE BRIS<br></i>
            <i>MS IA - 21/22<br></i>
        </h1>
    </center>
  </div>
  </div>
</div>

In [1]:
# Import packages
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import altair as alt
import panel as pn

import warnings
warnings.filterwarnings('ignore')

#PATH = 'data/'
PATH = './Names hints/'
primary_color = "#0072B5"
secondary_color = "#94EA84"

## Import des données

In [2]:
# Import working database
df = pd.read_csv(PATH+'dpt2020.csv', delimiter=";")
df = df.rename(columns={'preusuel': 'prenom', 'annais': 'annee'})

In [3]:
# Import prenoms database
df_pren = pd.read_csv(PATH+'prenoms.csv', delimiter=";")
df_pren.columns = pd.Index(['prenom', 'genre', 'langage', 'fréquence'])
df_pren['prenom'] = df_pren['prenom'].str.upper()

In [4]:
# Merge databases (original + names origin)
df = pd.merge(left=df, right=df_pren[['prenom', 'langage']],
              how='left', left_on='prenom', right_on='prenom')

## Fonctions de base (pré-traitement et visualisation avec Altair)

In [5]:
# Initialization of the main Dataframe
DF_BASE = df[df.annee != 'XXXX']
DF_BASE.annee = DF_BASE.annee.astype(int)
DF_BASE['prenom_sexe'] = DF_BASE.prenom + DF_BASE.sexe.astype(str)

In [6]:
# Aggregate data into global indicators for the comparative chart
def get_indicators(df_selection):
    df_temp = df_selection.reset_index()
    df_temp.annee = df_temp.annee - df_temp.annee.min()
    range_annee = df_temp.annee.max() - df_temp.annee.min()
    df_temp = df_temp.drop(columns='langage')

    results = []
    for prenom_sexe, df in df_temp.groupby('prenom_sexe'):
        results.append([prenom_sexe,
                        df.prenom.iloc[0],
                        df.sexe.iloc[0],
                        df.nombre.sum(),
                        df.annee.iloc[np.argmin(df.nombre.cumsum() <= (0.5 * df.nombre.sum()))],
                        np.polyfit(df.annee, df.nombre, 1)[0]])

    df_temp = pd.DataFrame(data=results, columns=['prenom_sexe', 'prenom',
                                                  'sexe', 'popularite',
                                                  'anciennete', 'tendance'])
    df_temp.anciennete = (df_temp.anciennete / range_annee) * 2 - 1
    
    return df_temp

In [7]:
# Define the visualisations
def plot_time(df_selection):
    # Compute the comparative indicators
    df_indicators = get_indicators(df_selection)
    
    # Selector for the names
    selection = alt.selection_multi(fields=['prenom_sexe'])
    color = alt.condition(selection,
                          alt.Color('prenom_sexe:N', legend=None),
                          alt.value('lightgray'))

    # Plot the temporal lines for names evolution
    lines = (alt.Chart(df_selection.reset_index())
             .mark_line(size=3)
             .encode(x=alt.X('annee:O',
                             axis=alt.Axis(labelOverlap=True,
                                           labelAngle=0,
                                           labelSeparation=10),
                             title='Année'),
                     y=alt.Y('nombre:Q',
                             scale=alt.Scale(type="log"),
                             title='Nombre de prénoms par an'), 
                     color=color,
                     strokeDash=alt.StrokeDash('sexe:N', title='Genre'),
                     tooltip=['prenom:N', 'nombre:Q', 'annee:O', 'langage'])
             .properties(width=800, height=400)
             .add_selection(selection)
             .interactive())

    # Plot the legend for the names
    legend = (alt.Chart(df_selection.reset_index())
              .mark_point().encode(x=alt.X('prenom:N',
                                           axis=alt.Axis(orient='bottom',
                                                         labelAngle=30,
                                                         labelOverlap=False,
                                                         title="")),
                                   color=color)
              .add_selection(selection)
              .properties(width=800,height=20))

    # Plot the comparative chart (cadran) 
    cadran = (alt.Chart(df_indicators)
              .mark_circle(filled=True)
              .encode(
                  x=alt.X('anciennete:Q',
                          title=["Essor du prénom", "⬅ Ancienne | Récente ⮕"],
                          scale=alt.Scale(domain=[-1, 1])),
                  y=alt.Y('tendance:Q', title=['Tendance du prénom']),
                  color=color,
                  size=alt.Size("popularite:Q",
                                scale=alt.Scale(type="log", zero=False,
                                                rangeMin=100,
                                                rangeMax=5000),
                                title=['Nombre cumulé', 'de prénoms'],
                                legend=alt.Legend(orient='none',
                                                  legendX=830,
                                                  legendY=560,)),
                  tooltip=['prenom:N', 'sexe:N', 'popularite:Q'])
              .properties(width=800, height=300)
              .add_selection(selection))

    # Plot the lines and texts on the comparative chart
    hline = alt.Chart(pd.DataFrame({'y': [0]})).mark_rule(color='black').encode(y='y:Q')
    vline = alt.Chart(pd.DataFrame({'x': [0]})).mark_rule(color='black').encode(x='x:Q')
    text1 = (alt.Chart({'values':[{'x': -1}]})
             .mark_text(text='⬆En hausse',align='left', baseline='middle', fontSize = 12)
             .encode(x='x:Q', y=alt.value(-10)))
    text2 = (alt.Chart({'values':[{'x': -1}]})
             .mark_text(text='⬇En baisse',align='left', baseline='middle', fontSize = 12)
             .encode(x='x:Q', y=alt.value(330)))
    
    return  lines & legend & (cadran+hline+vline+text1+text2)

In [8]:
# Filter the data in the main Dataframe regarding the "steadiness" criterion, set by the Users on the panel
from sklearn.linear_model import LinearRegression

def filter_steadiness_reg_lin(df_selection, critere_stabilite='Croissance rapide', seuil_linearite = 0.7, seuil_croissance = 0.5) :    
    """
    This function filters a dataframe (df_selection) so as to select curves that fit a criteria, given as argument ("critere_stabilite")
    Fits a linear regression on curves (one curve = occurences of a surname, aggregated on years) and classifies based on R2 coefficient and slope.
    Parameters : 
    - df_selection : dataframe to filter
    - critere_stabilite : "Tout", "Stable", "Croissance rapide", "Chute rapide", "Croissance constante", "Décroissance constante"
    - seuil_linearite : threshold that determines whether a curbe is linear or not (value to be compared to the R2 coefficient of a linear regression). Between 0 and 1.
    - seuil_croissance : threshold that determines whether a line is increasing, decreasing or flat.
    Returns a pandas dataframe.
    """
    if (critere_stabilite == 'Tout') :
        return df_selection
    
    df_save = df_selection
    agg_function = {'nombre': 'sum', 'prenom': 'first', 'sexe': 'first',  'langage': (lambda x: ', '.join(x.unique().astype(str)))}
    df_selection = df_selection.groupby(['prenom_sexe', 'annee']).agg(agg_function)

    df_selection = df_selection.reset_index()
    selected_names = []
    for prenom in np.unique(df_save.prenom_sexe) : 
        X = np.array(df_selection.loc[df_selection['prenom_sexe'] == prenom, 'annee']).reshape(-1, 1) #... # annee
        y = df_selection.loc[df_selection['prenom_sexe'] == prenom, 'nombre'] # nombre

        reg = LinearRegression().fit(X, y)
        score = reg.score(X, y)

        if (abs(score) < seuil_linearite) : # courbe, non lineaire
            coef = reg.coef_
            if (coef > seuil_croissance) : 
                if (critere_stabilite == 'Croissance rapide') :
                    selected_names.append(prenom)
            else : 
                if (critere_stabilite == 'Chute rapide') :
                    selected_names.append(prenom)

        elif (abs(score) > seuil_linearite) : # lineaire
            coef = reg.coef_
            if (abs(coef) < seuil_croissance) : 
                if (critere_stabilite == 'Stable') : 
                    selected_names.append(prenom)
            elif (coef > seuil_croissance) : 
                if (critere_stabilite == 'Croissance constante') :
                    selected_names.append(prenom)
            else : 
                if (critere_stabilite == 'Décroissance constante') :
                    selected_names.append(prenom)
    
    df_selection = df_selection[df_selection.prenom_sexe.isin(selected_names)]
    return df_selection


def lambda_reg_line(x) : 
    X = x.index[x.values == x.values].values
    if (len(X) == 1) : 
        return 1, 0
    X = X.reshape(-1, 1)
    y = x.values[x.values == x.values].reshape(-1, 1)
    reg = LinearRegression().fit(X, y)
    score = reg.score(X, y)
    coef = reg.coef_
    return score, coef[0][0]


def filter_steadiness_reg_lin_test(df_selection, critere_stabilite='Croissance rapide', seuil_linearite = 0.7, seuil_croissance = 0.5) :    
    ## autre version de la fonction, sans boucle "for" (utilisation de "pandas.pivot()"), pas plus efficace
    if (critere_stabilite == 'Tout') :
        return df_selection
    
    agg_function = {'nombre': 'sum', 'prenom': 'first', 'sexe': 'first',  'langage': (lambda x: ', '.join(x.unique().astype(str)))}
    df_selection = df_selection.groupby(['prenom_sexe', 'annee']).agg(agg_function)
    df_selection = df_selection.reset_index()
    df_selection = df_selection.pivot(index='prenom_sexe', columns='annee', values = 'nombre')#.fillna(0)
    df_selection['Regression'] = df_selection.apply(lambda x : lambda_reg_line(x), axis=1)
    df_selection['R2'] = df_selection['Regression'].apply(lambda x: x[0])
    df_selection['Coef'] = df_selection['Regression'].apply(lambda x: x[1])
    selected_names = []
    
    if (critere_stabilite == 'Stable') : 
        return df_selection.loc[(df_selection['R2'] > seuil_linearite) & (abs(df_selection['Coef']) < seuil_croissance),]
    
    elif (critere_stabilite == 'Croissance constante') : 
        return df_selection.loc[(df_selection['R2'] > seuil_linearite) & (df_selection['Coef'] > seuil_croissance),]
    
    elif (critere_stabilite == 'Décroissance constante') : 
        return df_selection.loc[(df_selection['R2'] > seuil_linearite) & (df_selection['Coef'] < -seuil_croissance),]
    
    elif (critere_stabilite == 'Croissance rapide') : 
        return df_selection.loc[(df_selection['R2'] < seuil_linearite) & (df_selection['Coef'] > seuil_croissance),]
    
    elif (critere_stabilite == 'Chute rapide') : 
        return df_selection.loc[(df_selection['R2'] < seuil_linearite) & (df_selection['Coef'] < -seuil_croissance),]
    

In [9]:
# Filter the data in the main Dataframe regarding the filters set by the Users on the panel
def select_data_time(prenom_regex='', annee_range=(1900, 2020),
                     popularite_range=(0, 1), sexe_range='Mixte', #stabilite='Tout',
                     langage_regex='', Nmax=20, view_fn=plot_time):
    
    sexe_dico = {'Homme': [1], 'Femme': [2], 'Mixte': [1, 2]}
    filter_sexe = DF_BASE.sexe.isin(sexe_dico[sexe_range])
    df_selection = DF_BASE[filter_sexe]

    filter_annee = ((DF_BASE.annee >= int(annee_range[0])) &
                    (DF_BASE.annee <= int(annee_range[1])))
    df_selection = df_selection[filter_annee]

    filter_prenom = df_selection.prenom.str.contains(prenom_regex.upper(), regex=True, na=False)
    df_selection = df_selection[filter_prenom]

    filter_langage = df_selection.langage.str.contains(langage_regex, regex=True, na=False)
    df_selection = df_selection[filter_langage]
    
    #df_selection = filter_steadiness_reg_lin(df_selection, critere_stabilite = stabilite)
    
    temp = df_selection.groupby('prenom_sexe').nombre.sum() # Count total number of names (popularity)
    filter_popularite = temp[temp.between(*temp.quantile(list(popularite_range)).tolist())].index
    df_selection = df_selection[df_selection.prenom_sexe.isin(filter_popularite)]

    # If more than Nmax values, keep the Nmax more frequent values
    filter_Nmax = list(df_selection.prenom_sexe.value_counts().index)[:Nmax]
    filter_Nmax = df_selection.prenom_sexe.isin(filter_Nmax)
    df_selection = df_selection[filter_Nmax]
    
    agg_function = {'sexe': 'first',
                    'prenom': 'first',
                    'nombre': 'sum',
                    'langage': (lambda x: ', '.join(x.unique().astype(str)))}

    df_selection = df_selection.groupby(['annee', 'prenom_sexe']).agg(agg_function)
    #df_selection = df_selection.droplevel(1)
    df_selection = df_selection.reset_index(level=1)
    df_selection.sexe = df_selection.sexe.replace({1: 'Homme', 2:'Femme'})
    
    return view_fn(df_selection)

In [10]:
# Test
prenom_regex = 'c'
annee_range = [1900, 2020]
popularite_range = [0, 1]
sexe_range = 'Mixte'
stabilite = 'Tout'
langage_regex = ''
Nmax = 20

plot_selection = select_data_time(prenom_regex, annee_range, popularite_range,
                                   sexe_range, langage_regex, Nmax)

#plot_selection = select_data_time(prenom_regex, annee_range, popularite_range,
#                                   sexe_range, stabilite, langage_regex, Nmax)

plot_selection

alt.VConcatChart(...)

## Définition du dashboard (avec Panel)

In [11]:
pn.extension()

In [12]:
prenom_regex = pn.widgets.TextInput(name='Prénom',
                                    placeholder='Taper la requête (regex possible)')

langage_regex = pn.widgets.TextInput(name='Langage',
                                    placeholder='Taper la requête (regex possible, en anglais)')

annee_range = pn.widgets.IntRangeSlider(name='Année(s)',
                                        start=1900, end=2020,
                                        value=(1900, 2020),
                                        step=1)

popularite_range = pn.widgets.RangeSlider(name='Popularité du prénom (sur une échelle de 0 à 1)',
                                        start=0, end=1,
                                        step=0.01)


sexe_range = pn.widgets.RadioButtonGroup(name='Genre',
                                         options=['Homme', 'Femme', 'Mixte'],
                                         value='Mixte',
                                         button_type='default')

#stabilite_buttons = pn.widgets.RadioButtonGroup(name='Stabilité', options=['Tout', 'Stable', 'Croissance rapide', 'Chute rapide',
                                                      #'Décroissance constante', 'Croissance constante'], value='Tout')

Nmax = pn.widgets.IntSlider(name='Nombre max de prénoms (les plus populaires)',
                                 start=1, end=50, step=1, value=20)

In [13]:
reactive_selection = pn.bind(select_data_time,
                            prenom_regex,
                            annee_range,
                            popularite_range,
                            sexe_range,
                            #stabilite_buttons,
                            langage_regex,
                            Nmax
                            )

In [14]:
reactive_selection

<function panel.depends._param_bind.<locals>.wrapped(*wargs, **wkwargs)>

In [15]:
text = ("## Vous hésitez pour le prénom de votre enfant ?\n" +
        "Explorer les prénoms données en France entre 1900 et 2020, et faîtes-vous une idée !\n\n" +
        "Choisissez les valeurs pour le filtrage")

widgets_text = pn.Row(prenom_regex, langage_regex)
widgets = pn.Column(pn.Row(sexe_range, Nmax),
                    widgets_text,
                    #stabilite_buttons,
                    annee_range,
                    popularite_range)
prenoms_plot = pn.Column(pn.panel(text), widgets, reactive_selection, max_width=800)

## Lancer la visualisation !

In [16]:
prenoms_plot.show()

Launching server at http://localhost:61510


*Note* : en plus des graphiques affichés lors de l'exécution, nous avons essayé d'ajouter des boutons supplémentaires permettant de filtrer sur la popularité des prénoms au cours du temps (filtre sur la "forme" des courbes : stable, mouvementée, globalement croissante / dcéroissante...) 

Pour ce faire, nous avons fait une régression linéaire sur les courbes traçant l'évolution du nombre d'occurence de chaque prénom (nombre national), et observé les *coefficient de détermination* (R2) et *coefficient directeur* obtenus.

- Si le coefficient R2 est supérieur à un certain seuil, cela signifie que le droite de régression est suffisamment "proche" des données observée, et donc que l'évolution est *linéaire* ("stable"). Le coefficient directeur permet ensuite d'indiquer une *tendance positive* (prénom de plus en plus populaire) ou *négative* (perte de popularité).
- Inversement, un coefficient R2 trop faible indique une évolution non linéaire (souvent convexe / concave), indiquant que le prénom a été populaire (resp. impopulaire) pendant une période, avant de perdre (resp. reprendre) en popularité.

Ces deux critères (R2 et pente) nous ont permis d'obtenir des résultats intéressants (les courbes sélectionnées correspondaient effectivement au critère sélectionné), mais nous avons commenté le code à cause du temps d'éxecution. Nous avons testé d'autres approches (notamment basées sur la variance et les écarts inter-quartiles), moins coûteuses, mais qui ne nous ont pas permis d'obtenir d'aussi bons résultats.